<a href="https://colab.research.google.com/github/hoang2111/grinnlo.github.io/blob/main/nba_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install nba_api
%pip install pandas

Imports

In [3]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder

from datetime import datetime
import pandas as pd

DATA CLEANING


In [4]:
gamefinder = leaguegamefinder.LeagueGameFinder()
games = gamefinder.get_data_frames()[0]

nba_teams_list = teams.get_teams()
teams = [item['full_name'] for item in nba_teams_list]

games_17to19 = games[games['TEAM_NAME'].isin(teams)]

games_17to19 = games_17to19[(games.SEASON_ID.str[-4:] >= '2017') & (games.SEASON_ID.str[-4:] <= '2022')]

games_17to19['SEASON_ID'] = games_17to19['SEASON_ID'].str[-4:].astype(int)

games_17to19

/tmp/ipython-input-1806964003.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  games_17to19 = games_17to19[(games.SEASON_ID.str[-4:] >= '2017') & (games.SEASON_ID.str[-4:] <= '2022')]


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
7026,2022,1610612743,DEN,Denver Nuggets,0042200405,2023-06-12,DEN vs. MIA,W,240,94,...,0.565,11,46,57,21,6,7,14,13,5.0
7027,2022,1610612748,MIA,Miami Heat,0042200405,2023-06-12,MIA @ DEN,L,240,89,...,0.875,11,33,44,18,9,7,8,21,-5.0
7028,2022,1610612748,MIA,Miami Heat,0042200404,2023-06-09,MIA vs. DEN,L,240,95,...,0.850,8,29,37,23,2,3,14,19,-13.0
7029,2022,1610612743,DEN,Denver Nuggets,0042200404,2023-06-09,DEN @ MIA,W,242,108,...,0.762,5,29,34,26,11,7,6,18,13.0
7030,2022,1610612743,DEN,Denver Nuggets,0042200403,2023-06-07,DEN @ MIA,W,238,109,...,0.815,13,45,58,28,3,5,13,18,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23130,2017,1610612737,ATL,Atlanta Hawks,0011700004,2017-10-01,ATL @ MIA,L,241,90,...,0.815,10,32,42,18,14,1,16,25,-6.0
23131,2017,1610612750,MIN,Minnesota Timberwolves,0011700002,2017-09-30,MIN @ LAL,W,240,108,...,0.800,14,35,49,28,7,3,17,21,9.0
23132,2017,1610612744,GSW,Golden State Warriors,0011700001,2017-09-30,GSW vs. DEN,L,240,102,...,0.833,17,36,53,17,13,4,14,28,-6.0
23133,2017,1610612743,DEN,Denver Nuggets,0011700001,2017-09-30,DEN @ GSW,W,240,108,...,0.676,9,40,49,23,6,0,18,24,6.0


In [5]:
numeric_columns = games_17to19.select_dtypes(include=['number']).columns.tolist()
numeric_columns

['SEASON_ID',
 'TEAM_ID',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PLUS_MINUS']

In [4]:
games_17to19 = games_17to19.sort_values("GAME_DATE")
games_17to19 = games_17to19.reset_index(drop=True)

In [5]:
def add_target(team):
  team["target"] = team["WL"].shift(-1)
  return team

games_17to19 = games_17to19.groupby("TEAM_NAME", group_keys=False).apply(add_target)

/tmp/ipython-input-2870168298.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  games_17to19 = games_17to19.groupby("TEAM_NAME", group_keys=False).apply(add_target)


In [6]:
games_17to19['target'] = games_17to19['target'].map({'W': 1, 'L': 0})

In [7]:
games_17to19.loc[pd.isnull(games_17to19['target']), 'target'] = 2
games_17to19

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,target
0,2017,1610612747,LAL,Los Angeles Lakers,0011700002,2017-09-30,LAL vs. MIN,L,241,99,...,13,35,48,27,10,6,17,29,-9.0,0.0
1,2017,1610612750,MIN,Minnesota Timberwolves,0011700002,2017-09-30,MIN @ LAL,W,240,108,...,14,35,49,28,7,3,17,21,9.0,1.0
2,2017,1610612743,DEN,Denver Nuggets,0011700001,2017-09-30,DEN @ GSW,W,240,108,...,9,40,49,23,6,0,18,24,6.0,1.0
3,2017,1610612744,GSW,Golden State Warriors,0011700001,2017-09-30,GSW vs. DEN,L,240,102,...,17,36,53,17,13,4,14,28,-6.0,0.0
4,2017,1610612761,TOR,Toronto Raptors,0011700003,2017-10-01,TOR vs. LAC,W,240,121,...,9,35,44,22,14,3,21,34,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15480,2022,1610612743,DEN,Denver Nuggets,0042200403,2023-06-07,DEN @ MIA,W,238,109,...,13,45,58,28,3,5,13,18,15.0,1.0
15481,2022,1610612743,DEN,Denver Nuggets,0042200404,2023-06-09,DEN @ MIA,W,242,108,...,5,29,34,26,11,7,6,18,13.0,1.0
15482,2022,1610612748,MIA,Miami Heat,0042200404,2023-06-09,MIA vs. DEN,L,240,95,...,8,29,37,23,2,3,14,19,-13.0,0.0
15483,2022,1610612748,MIA,Miami Heat,0042200405,2023-06-12,MIA @ DEN,L,240,89,...,11,33,44,18,9,7,8,21,-5.0,2.0


In [8]:
games_17to19 = games_17to19.dropna(subset=['WL']).reset_index(drop=True)
games_17to19 = games_17to19.dropna(subset=['FG_PCT']).reset_index(drop=True)
games_17to19 = games_17to19.dropna(subset=['FG3_PCT']).reset_index(drop=True)
games_17to19

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,target
0,2017,1610612750,MIN,Minnesota Timberwolves,0011700002,2017-09-30,MIN @ LAL,W,240,108,...,14,35,49,28,7,3,17,21,9.0,1.0
1,2017,1610612743,DEN,Denver Nuggets,0011700001,2017-09-30,DEN @ GSW,W,240,108,...,9,40,49,23,6,0,18,24,6.0,1.0
2,2017,1610612744,GSW,Golden State Warriors,0011700001,2017-09-30,GSW vs. DEN,L,240,102,...,17,36,53,17,13,4,14,28,-6.0,0.0
3,2017,1610612747,LAL,Los Angeles Lakers,0011700002,2017-09-30,LAL vs. MIN,L,241,99,...,13,35,48,27,10,6,17,29,-9.0,0.0
4,2017,1610612761,TOR,Toronto Raptors,0011700003,2017-10-01,TOR vs. LAC,W,240,121,...,9,35,44,22,14,3,21,34,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15479,2022,1610612748,MIA,Miami Heat,0042200403,2023-06-07,MIA vs. DEN,L,240,94,...,10,23,33,20,7,3,4,22,-15.0,0.0
15480,2022,1610612743,DEN,Denver Nuggets,0042200404,2023-06-09,DEN @ MIA,W,242,108,...,5,29,34,26,11,7,6,18,13.0,1.0
15481,2022,1610612748,MIA,Miami Heat,0042200404,2023-06-09,MIA vs. DEN,L,240,95,...,8,29,37,23,2,3,14,19,-13.0,0.0
15482,2022,1610612743,DEN,Denver Nuggets,0042200405,2023-06-12,DEN vs. MIA,W,240,94,...,11,46,57,21,6,7,14,13,5.0,2.0


FEATURE SELECTION

In [9]:
%pip install scikit-learn

In [10]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

In [11]:
rr = RidgeClassifier(alpha=1e12)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr, n_features_to_select=10, direction="forward", cv=split)

In [12]:
numeric_columns = games_17to19.select_dtypes(include=['number']).columns.tolist()
numeric_columns = numeric_columns[1:(len(numeric_columns) - 1)]
numeric_columns

['TEAM_ID',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PLUS_MINUS']

In [13]:
sfs.fit(games_17to19[numeric_columns], games_17to19["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1000000000000.0),
                          n_features_to_select=10)

In [14]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler()

numeric_cols_array = np.array(numeric_columns)
games_17to19[numeric_cols_array] = scaler.fit_transform(games_17to19[numeric_cols_array])

predictor = list(numeric_cols_array[sfs.get_support()])

In [15]:
games_17to19['SEASON_ID']

,SEASON_ID
0,2017
1,2017
2,2017
3,2017
4,2017
...,...
15479,2022
15480,2022
15481,2022
15482,2022


In [16]:
def backtest(data, model, predictors, start=2, step=1):
  all_predictions = []

  seasons = sorted(data['SEASON_ID'].unique())

  for i in range(start, len(seasons), step):

    current_season = seasons[i]

    train = data[data["SEASON_ID"] < current_season]
    test = data[data["SEASON_ID"] == current_season]

    model.fit(train[predictors], train['target'])

    preds = model.predict(test[predictors])

    preds = pd.Series(preds, index=test.index)

    combined = pd.concat( [test["target"], preds], axis=1)
    combined.columns = ["actual", "prediction"]
    all_predictions.append(combined)
  return pd.concat(all_predictions)


In [17]:
predictions = backtest(games_17to19, rr, predictor)

In [18]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions['actual'] != 2]
accuracy_score(predictions['actual'], predictions['prediction'])

0.5

In [19]:
games_rolling = games_17to19[list(numeric_columns) + ['WL', 'TEAM_NAME', 'SEASON_ID']].copy()

games_rolling['WL'] = games_rolling['WL'].map({'W': 1, 'L': 0})

games_rolling

,TEAM_ID,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,WL,TEAM_NAME,SEASON_ID
0,0.448276,0.303448,0.418803,0.363636,0.426230,0.351675,0.178571,0.180328,0.310433,0.627907,...,0.529412,0.476190,0.318182,0.15,0.50000,0.384615,0.561644,1,Minnesota Timberwolves,2017
1,0.206897,0.303448,0.418803,0.340909,0.295082,0.418660,0.428571,0.311475,0.519084,0.511628,...,0.529412,0.357143,0.272727,0.00,0.53125,0.461538,0.541096,1,Denver Nuggets,2017
2,0.241379,0.303448,0.367521,0.409091,0.524590,0.339713,0.107143,0.393443,0.082697,0.441860,...,0.607843,0.214286,0.590909,0.20,0.40625,0.564103,0.458904,0,Golden State Warriors,2017
3,0.344828,0.310345,0.341880,0.454545,0.491803,0.411483,0.142857,0.344262,0.141221,0.255814,...,0.509804,0.452381,0.454545,0.30,0.50000,0.589744,0.438356,0,Los Angeles Lakers,2017
4,0.827586,0.303448,0.529915,0.409091,0.245902,0.552632,0.357143,0.557377,0.254453,0.720930,...,0.431373,0.333333,0.636364,0.15,0.62500,0.717949,0.554795,1,Toronto Raptors,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15479,0.379310,0.303448,0.299145,0.295455,0.475410,0.241627,0.357143,0.426230,0.328244,0.325581,...,0.215686,0.285714,0.318182,0.15,0.09375,0.410256,0.397260,0,Miami Heat,2022
15480,0.206897,0.317241,0.418803,0.409091,0.262295,0.538278,0.464286,0.311475,0.564885,0.348837,...,0.235294,0.428571,0.500000,0.35,0.15625,0.307692,0.589041,1,Denver Nuggets,2022
15481,0.379310,0.303448,0.307692,0.318182,0.245902,0.430622,0.250000,0.262295,0.335878,0.372093,...,0.294118,0.357143,0.090909,0.15,0.40625,0.333333,0.410959,0,Miami Heat,2022
15482,0.206897,0.303448,0.299145,0.386364,0.344262,0.437799,0.142857,0.311475,0.156489,0.279070,...,0.686275,0.309524,0.272727,0.35,0.40625,0.179487,0.534247,1,Denver Nuggets,2022


In [20]:
# List of stats we want to average (Points, FG%, etc.)
# Include 'WL' since we mapped it to 1s and 0s
stats_to_roll = list(numeric_columns) + ['WL']

def find_team_average(team):
    # Only roll the numeric statistics
    rolling = team[stats_to_roll].rolling(10).mean()
    return rolling

# Apply the grouping
games_rolling_stats = games_rolling.groupby(['TEAM_NAME', 'SEASON_ID'], group_keys=False).apply(find_team_average)

/tmp/ipython-input-1078375096.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  games_rolling_stats = games_rolling.groupby(['TEAM_NAME', 'SEASON_ID'], group_keys=False).apply(find_team_average)


In [21]:
# 1. Rename columns to indicate they are 10-game averages
games_rolling_stats.columns = [f"{col}_10" for col in games_rolling_stats.columns]

# 2. Combine with the original team/season/date info
games_rolling_final = pd.concat([games_rolling, games_rolling_stats], axis=1)

# 3. Drop the 'NaN' rows (the first 9 games of every season for every team)

In [22]:
games_rolling_final_cleaned = games_rolling_final.drop(columns=['SEASON_ID'])
games_17to19 = pd.concat([games_17to19, games_rolling_final_cleaned], axis=1)
games_17to19 = games_17to19.dropna()

In [24]:
def shift_col(team, col_name):
  next_col = team[col_name].shift(-1)
  return next_col
def add_col(df, col_name) :
  return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name) )
games_17to19["home_next"] = add_col(games_17to19, "home")
games_17to19["team_opp_next"] = add_col(games_17to19, "team_opp" )
games_17to19["date_next"] = add_col(games_17to19, "date")



0.5013768036127326